# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.11.4.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [14]:
import sys
import os

# Make sure we can import the aimakerspace package
if 'aimakerspace' not in sys.modules:
	# Get the absolute path to the parent directory containing the aimakerspace package
	project_root = os.path.abspath(os.path.join(os.path.dirname('__file__')))
	if project_root not in sys.path:
		sys.path.append(project_root)
	print(f"Added {project_root} to sys.path")

#from dotenv import load_dotenv
#load_dotenv(override=True)	

In [15]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [16]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using blogs from PMarca (Marc Andreessen) as our sample data. This data is largely irrelevant as we want to focus on the mechanisms of RAG, which includes out data's shape and quality - but not specifically what the contents of the data are. 


In [17]:
text_loader = TextFileLoader("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
len(documents)

1

In [18]:
print(documents[0][:100])


The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horow


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [19]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

373

Let's take a look at some of the documents we've managed to split.

In [20]:
split_documents[0:1]

['\ufeff\nThe Pmarca Blog Archives\n(select posts from 2007-2009)\nMarc Andreessen\ncopyright: Andreessen Horowitz\ncover design: Jessica Hagy\nproduced using: Pressbooks\nContents\nTHE PMARCA GUIDE TO STARTUPS\nPart 1: Why not to do a startup 2\nPart 2: When the VCs say "no" 10\nPart 3: "But I don\'t know any VCs!" 18\nPart 4: The only thing that matters 25\nPart 5: The Moby Dick theory of big companies 33\nPart 6: How much funding is too little? Too much? 41\nPart 7: Why a startup\'s initial business plan doesn\'t\nmatter that much\n49\nTHE PMARCA GUIDE TO HIRING\nPart 8: Hiring, managing, promoting, and Dring\nexecutives\n54\nPart 9: How to hire a professional CEO 68\nHow to hire the best people you\'ve ever worked\nwith\n69\nTHE PMARCA GUIDE TO BIG COMPANIES\nPart 1: Turnaround! 82\nPart 2: Retaining great people 86\nTHE PMARCA GUIDE TO CAREER, PRODUCTIVITY,\nAND SOME OTHER THINGS\nIntroduction 97\nPart 1: Opportunity 99\nPart 2: Skills and education 107\nPart 3: Where to go and wh

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [21]:
import os
from dotenv import load_dotenv
import openai
from getpass import getpass
_ = load_dotenv(override=True)

#openai.api_key = os.getenv("OPENAI_API_KEY",getpass("OpenAI API Key: "))
#os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension? **Answer:** Yes, we can use dimensions parameter, I have modified `async_get_embeddings` method of `EmbeddingModel()` to accept dimensions parameter
2. What technique does OpenAI use to achieve this? **Answer:** OpenAI has trained there embedding models  with dimensionality reduction technique called [Matryoshka Representation Learning](https://arxiv.org/abs/2205.13147).

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #2!

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [22]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings? **Answer:**  Using async will run tasks concurrently in event loop, when one task awaitng on I/O or network then next task will run. As communication with OpenAI is network bound that's why running async will result in better throughput.


> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [23]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)


[('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales executive. The CEO who used to be\nin marketing who has a weak marketing executive.\nI call this the “Michael Eisner Memorial Weak Executive Problem” — aaer the CEO of Disney who had previously been a brilliant TV network executive. When he bought ABC at Disney, it\npromptly fell to fourth place. His response? “If I had an extra\ntwo days a week, I could turn around ABC myself.” Well, guess\nwhat, he didn’t have an extra two days a week.\nA CEO — or a startup founder — oaen has a hard time letting\ngo of the function that brought him to the party. The result: you\nhire someone weak into the executive role for that function so\nthat you can continue to b

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4o-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs? **Answer:** We can use temprature parameter and lower value will help to achieve more reproducible outputs. We can pass `temperature` parameter to `run` method of `ChatOpenAI()`.

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

### Creating and Prompting OpenAI's `gpt-4o-mini`!

Let's tie all these together and use it to prompt `gpt-4o-mini`!

In [29]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
# user_prompt_template = "{content}"
# user_role_prompt = UserRolePrompt(user_prompt_template)
# system_prompt_template = (
#     "You are an expert in {expertise}, you always answer in a kind way."
# )
# system_role_prompt = SystemRolePrompt(system_prompt_template)

# messages = [
#     system_role_prompt.create_message(expertise="Python"),
#     user_role_prompt.create_message(
#         content="What is the best way to write a loop?"
#     ),
# ]

# response = chat_openai.run(messages, temperature=0.5)

In [25]:
# print(response)

## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [31]:
RAG_PROMPT_TEMPLATE = """ \
Use the provided context to answer the user's query.

You may not answer the user's query unless there is specific context in the following text.

If you do not know the answer, or cannot answer, please respond with "I don't know".
"""

rag_prompt = SystemRolePrompt(RAG_PROMPT_TEMPLATE)

USER_PROMPT_TEMPLATE = """ \
Context:
{context}

User Query:
{user_query}
"""


user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)

        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = rag_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)

        return {"response" : self.llm.run([formatted_system_prompt, formatted_user_prompt]), "context" : context_list}

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response? We can instruct LLM to use **Think though your response step by step.** to have more thoughtful, detailed response.

What is that strategy called? The strategy called **Chain of Thought Prompting**

> NOTE: You can look through ["Accessing GPT-3.5-turbo Like a Developer"](https://colab.research.google.com/drive/1mOzbgf4a2SP5qQj33ZxTz2a01-5eXqk2?usp=sharing) for an answer to this question if you get stuck!

In [ ]:
# retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
#     vector_db_retriever=vector_db,
#     llm=chat_openai
# )

In [ ]:
# retrieval_augmented_qa_pipeline.run_pipeline("What is the Michael Eisner Memorial Weak Executive Problem?")


{'response': 'The Michael Eisner Memorial Weak Executive Problem refers to the tendency of a CEO or startup founder to hire a weak executive in their former specialty area, often out of a desire to maintain control over that function. This can happen when a CEO, who has expertise in a specific area like product management, sales, or marketing, selects someone less competent for that role to ensure they can remain prominent and influential (“the man”) in that specific function. This problem highlights the risk of allowing personal biases and insecurities to influence hiring decisions, which can lead to poor organizational performance. The term is based on the example of Michael Eisner, who struggled to turn around ABC after acquiring it, despite his previous success as a TV network executive.',
 'context': [('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oae

### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database

While these are suggestions, you should feel free to make whatever augmentations you desire! 

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

In [32]:
from collections import defaultdict
import numpy as np
from typing import List, Tuple, Dict, Any, Optional
from aimakerspace.openai_utils.chatmodel import ChatOpenAI
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)
import numpy as np

USER_PROMPT_TEMPLATE = """ \
Context:
{context}

User Query:
{user_query}
"""

# implemented euclidean distance function
def euclidean_distance(vector_a: np.array, vector_b: np.array) -> float:
    return  -1 * (np.linalg.norm(vector_a - vector_b))
        
        

# 1. Create enhanced RAG pipeline with improved prompt and new distance metric support
class EnhancedRAGPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        
        # Enhanced system prompt with Chain of Thought reasoning
        self.system_prompt = SystemRolePrompt("""
        Use the provided context to answer the user's query.
        
        Think through your answer step by step:
        1. First, analyze the relevant information from the provided context
        2. Identify the key points that address the user's query
        3. Organize this information into a clear, structured response
        4. If multiple pieces of context provide different perspectives, synthesize them
        
        You may not answer the user's query unless there is specific context in the following text.
        If you do not know the answer, or cannot answer, please respond with "I don't know".
        """)
        
        self.user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)
    
    def run_pipeline(self, user_query: str, k: int = 4, metric: str='cosine') -> dict:
        """
        Run the enhanced RAG pipeline with choice of similarity metric
        
        Args:
            user_query: The user's question
            k: Number of context items to retrieve
            metric: "cosine" or "euclidean" for distance calculation
        """
        context_list = []

        if metric == 'euclidean':
            context_list = self.vector_db_retriever.search_by_text(user_query, k=k, distance_measure=euclidean_distance)
        else:
            context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        # Format context, including any metadata if available
        context_prompt = ""
        for text,_ in context_list:
            
            context_prompt += f"{text}\n\n"
                
        formatted_system_prompt = self.system_prompt.create_message()
        formatted_user_prompt = self.user_prompt.create_message(
            user_query=user_query, 
            context=context_prompt
        )
        
        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt], temperature=0.2),
            "context": context_list
        }


chat_openai = ChatOpenAI()
# Create enhanced RAG pipeline
enhanced_rag_pipeline = EnhancedRAGPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)



In [33]:
import os
from glob import glob

# Function to get all pdf, txt, and md files in a directory
def load_file_list(directory="data", extensions=["pdf", "txt", "md"]):
    """
    Lists all files with specified extensions in the given directory.
    
    Args:
        directory (str): Directory to search for files
        extensions (list): List of file extensions to include
    
    Returns:
        list: List of file paths found
    """
    all_files = []
    
    # Make sure the directory exists
    if not os.path.exists(directory):
        print(f"Directory '{directory}' does not exist")
        return all_files
    
    # Get all files with specified extensions
    for ext in extensions:
        pattern = os.path.join(directory, f"*.{ext}")
        files = glob(pattern)
        all_files.extend(files)
    
    return sorted(all_files)

# Function to display files with selection options
def display_file_info():
    files = load_file_list()
    
    if not files:
        return "No files found in data directory"
    
    result = "### Available Files\n\n"
    for i, file_path in enumerate(files):
        file_name = os.path.basename(file_path)
        file_size = os.path.getsize(file_path) / 1024  # Size in KB
        result += f"{i+1}. **{file_name}** ({file_size:.1f} KB)\n"
    
    return result



In [34]:
# from IPython.display import display, Markdown
# display(Markdown(display_file_info()))

In [35]:
import json
def parse_json_output(json_output: str) -> dict:
    """
    Parses JSON output from the model and returns a dictionary.
    
    Args:
        json_output (str): JSON output as a string
        
    Returns:
        dict: Parsed JSON data
    """
    try:
        return json.loads(json_output)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return {}
    
def get_summary_and_questions(content: str) -> Tuple[List[str], str]:
    """
    Extracts summary and questions from the content.
    
    Args:
        content (str): Content to analyze
        
    Returns:
        Tuple[List[str], str]: List of questions and summary
    """
    # Use the chat model to get the summary and questions

    user_prompt_template = "Analyze and generate RAG questions\n\n{content}"
    user_role_prompt = UserRolePrompt(user_prompt_template)
    system_prompt_template = """
    You are an expert AI researcher who specializes in reading and analyzing academic papers.
    Your task is to help users understand complex research papers by explaining concepts clearly, identifying key methodologies, and answering questions about the paper's findings. When presented with an abstract or content from a paper, you should be able to extract the main research questions, methodologies, results, and limitations.
    Your responses should be well-structured, accurate, and accessible to users with various levels of technical knowledge.
    You should also be able to generate questions based on the content provided to you, which can help users think critically about the material.
    Generate output in JSON format with the following keys:
    - "questions": A list of questions generated from the content.
    - "summary": A brief summary of the content.

    The output should be in the following format:
    {output}
    """

    example_output = """
    {
        "questions": [
            "What are the main findings of the research?",
            "What methodologies were used in the study?",
            "What are the limitations of the research?",
            "How does this research contribute to the field?"
        ],
        "summary": "The research explores the impact of X on Y, using methodology Z. The findings suggest that A leads to B, with limitations including C and D. This contributes to the field by providing insights into E."
    }
    """
    

    # Set up the system prompt with the example output
    system_role_prompt = SystemRolePrompt(system_prompt_template)

    messages = [
        system_role_prompt.create_message(output=example_output),
        user_role_prompt.create_message(content=content)
    ]

    # Get response from the model with JSON format
    response = chat_openai.run(messages, temperature=0.1,response_format={"type": "json_object"})
    
    # Print the response for debugging  
    print(f"Response: {response}")  
    # Parse the JSON output
    parsed_response = parse_json_output(response)
    
    return parsed_response.get("questions", []), parsed_response.get("summary", "")

In [36]:
# create dictionary to vector db by dataset name
questions, summary = get_summary_and_questions(documents[0][:2000])
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

Response: {
    "questions": [
        "What are the key reasons provided for why one might choose not to start a startup?",
        "What insights does Marc Andreessen offer regarding venture capitalists' responses?",
        "How does Andreessen suggest one should approach hiring in startups?",
        "What psychological biases does the blog address in relation to entrepreneurship?",
        "What are the main themes covered in the Pmarca Guide to Startups?"
    ],
    "summary": "The Pmarca Blog Archives, authored by Marc Andreessen, compiles insights and experiences related to startups, hiring, and entrepreneurship. It includes various guides addressing why one might avoid starting a startup, how to navigate venture capital, effective hiring practices, and psychological factors influencing entrepreneurs. The content reflects Andreessen's extensive experience with multiple startups, providing valuable lessons and observations for aspiring entrepreneurs."
}


In [37]:
vector_db_by_dataset = {
    "data/PMarcaBlogs.txt": {'db':vector_db,'summary':summary,'questions':questions}
}

In [38]:
def load_dataset_from_selected_file(file_path: str) -> None:
    """
    Loads the selected file and returns its content.
    
    Args:
        file_path (str): Path to the file
        
    
    """
    if not os.path.exists(file_path):
        print(f"File '{file_path}' does not exist")
    
    global summary
    global questions 

    if file_path in vector_db_by_dataset:
        questions = vector_db_by_dataset[file_path]['questions']
        summary = vector_db_by_dataset[file_path]['summary']
        enhanced_rag_pipeline.vector_db_retriever = vector_db_by_dataset[file_path]['db']
    else:
        
        documents = []
        # Load the selected file into the vector database
        if file_path.endswith('.pdf'):
            from aimakerspace.PDFTextExtractor import PDFTextExtractor
            pdf_extractor = PDFTextExtractor(file_path)
            documents = pdf_extractor.split_documents()
        else:
            text_loader = TextFileLoader(file_path)
            documents = text_loader.load_documents()
            
        text_splitter = CharacterTextSplitter()
        split_documents = text_splitter.split_texts(documents)
        # Build the vector database from the split documents
        vector_db = VectorDatabase()
        vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))
        vector_db_by_dataset[file_path] = vector_db
        
        enhanced_rag_pipeline.vector_db_retriever = vector_db
        
         # Get summary and questions
        questions, summary = get_summary_and_questions(documents[0][:2000])
        # Store in the dictionary
        vector_db_by_dataset[file_path] = {'db':vector_db,'summary':summary,'questions':questions}
        

In [ ]:
import ipywidgets as widgets
from IPython.display import clear_output, display, Markdown

# === Function to create consistent widget styles ===
def create_widget_layout(width='300px'):
    return widgets.Layout(width=width, margin='5px 0')

def create_description_style():
    return {'description_width': '80px'}

# === UI Components Creation ===
# File selection
file_dropdown = widgets.Dropdown(
    options=load_file_list(),
    value="data/PMarcaBlogs.txt",
    description='Select File:',
    disabled=False,
    layout=create_widget_layout(),
    style=create_description_style()
)

# Search settings
k_slider = widgets.IntSlider(
    value=4, min=1, max=10, step=1,
    description='Top K:',
    tooltip='Select number of context chunks to retrieve',
    continuous_update=False,
    layout=create_widget_layout(),
    style=create_description_style()
)

metric_dropdown = widgets.Dropdown(
    options=['cosine', 'euclidean'],
    value='cosine',
    description='Metric:',
    layout=create_widget_layout(),
    style=create_description_style()
)

# Chat input
question_input = widgets.Text(
    value='',
    placeholder='Type your question here...',
    description='',
    continuous_update=False,
    layout=widgets.Layout(width='90%')
)

send_button = widgets.Button(
    description='',
    button_style='primary',
    tooltip='Send message',
    icon='paper-plane',
    layout=widgets.Layout(width='50px')
)

# Output areas
output_area = widgets.Output()
chat_history = widgets.Output(
    layout=widgets.Layout(
        height='500px', 
        border='1px solid #ddd', 
        padding='8px', 
        overflow_y='auto',
        margin='10px 0'
    )
)
summary_output = widgets.Output()
questions_output = widgets.Output()

# === Display Functions ===
def display_summary():
    with summary_output:
        clear_output(wait=True)
        display(Markdown("### Summary:"))
        display(Markdown(summary))

def display_questions():
    with questions_output:
        clear_output(wait=True)
        display(Markdown("### Suggested Questions:"))
        questions_box = widgets.HBox(layout=widgets.Layout(flex_wrap='wrap'))
        
        def on_question_click(b):
            question_input.value = b.description
            
        for question in questions:
            button = widgets.Button(
                description=question, 
                button_style='info', 
                layout=widgets.Layout(width='auto', margin='2px')
            )
            button.on_click(on_question_click)
            questions_box.children += (button,)
        display(questions_box)

# === Event Handlers ===
def on_send_click(b):
    question = question_input.value.strip()
    if not question:
        return
        
    with chat_history:
        clear_output(wait=True)
        display(Markdown(f"<div style='text-align: right; margin: 8px 0;'><span style='background-color: #e1f5fe; padding: 8px 12px; border-radius: 15px; display: inline-block;'><b>You:</b> {question}</span></div>"))
        display(Markdown("<div style='margin: 8px 0;'><span style='background-color: #f1f1f1; padding: 8px 12px; border-radius: 15px; display: inline-block;'><b>Assistant:</b> Thinking...</span></div>"))
    
    # Get response using the RAG pipeline
    result = enhanced_rag_pipeline.run_pipeline(
        question, 
        k=k_slider.value, 
        metric=metric_dropdown.value
    )
    
    with chat_history:
        clear_output(wait=True)
        display(Markdown(f"<div style='text-align: right; margin: 8px 0;'><span style='background-color: #e1f5fe; padding: 8px 12px; border-radius: 15px; display: inline-block;'><b>You:</b> {question}</span></div>"))
        display(Markdown(f"<div style='margin: 8px 0;'><span style='background-color: #f1f1f1; padding: 8px 12px; border-radius: 15px; display: inline-block;'><b>Assistant:</b> {result['response']}</span></div>"))
        
        sources_html = "<details><summary><b>Sources</b></summary><div style='margin-left: 20px; font-size: 0.9em;'>"
        for i, (text, score) in enumerate(result['context']):
            preview = text[:150].replace('\n', ' ') + "..." if len(text) > 150 else text
            sources_html += f"<p><b>Source {i+1}</b> (score: {score:.4f}):<br><code>{preview}</code></p>"
        sources_html += "</div></details>"
        display(Markdown(sources_html))
    
    question_input.value = ''

def load_selected_file(change):
    selected_file = file_dropdown.value
    if selected_file:
        with output_area:
            clear_output(wait=True)
            display(Markdown(f"Loading **{selected_file}**..."))
            load_dataset_from_selected_file(selected_file)
            clear_output(wait=True)
            display(Markdown(f"Loaded **{selected_file}**!"))
        
        display_summary()
        display_questions()
        
        with chat_history:
            clear_output()

# === Connect events ===
send_button.on_click(on_send_click)
question_input.on_submit(lambda _: on_send_click(None))
file_dropdown.observe(load_selected_file, names='value')

# === UI Layout ===
# Header
header = widgets.HTML("<h2>Chat with Your Documents</h2>")

# Group settings in accordion
settings_box = widgets.VBox([k_slider, metric_dropdown])
settings = widgets.Accordion(
    children=[settings_box], 
    titles=('Search Settings',)
)

# Chat input area
input_area = widgets.HBox(
    [question_input, send_button], 
    layout=widgets.Layout(width='100%', margin='5px 0')
)

# Main container
widgets_container = widgets.VBox([
    header,
    file_dropdown,
    widgets.HBox([
        widgets.VBox([summary_output, questions_output], layout=widgets.Layout(width='100%')),
    ], layout=widgets.Layout(margin='10px 0')),
    settings,
    chat_history,
    input_area,
], layout=widgets.Layout(padding='10px'))

# Initial UI setup
display_summary()
display_questions()
display(widgets_container)

/tmp/ipykernel_66700/1646829267.py:146: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  question_input.on_submit(lambda _: on_send_click(None))
